In [1]:
import requests
import json
import os
import math
import time
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import matplotlib.pyplot as plt

from shapely.geometry import mapping, shape, Point, Polygon, LineString
from folium import plugins
from folium.plugins import MarkerCluster, HeatMap

In [3]:
def get_access_token(consumer_key, consumer_secret) -> None:
    temp_res = requests.get(
        url="https://sgisapi.kostat.go.kr/OpenAPI3/auth/authentication.json",
        params={
            "consumer_key": consumer_key,
            "consumer_secret": consumer_secret
        }
    )

    access_token = temp_res.json()["result"]["accessToken"]

    return access_token

consumer_key = "0557bd5252b34142911d"
consumer_secret = "d90d86911924440786d4"

print('access token: ', get_access_token(consumer_key, consumer_secret))

access token:  cd91f03d-5596-4a3d-a279-31a09d25902b


In [4]:
def get_si_info_list(consumer_key, consumer_secret, cd) -> None:
    ACCESS_TOKEN=get_access_token(consumer_key, consumer_secret)
    PARAMS={
        "accessToken": ACCESS_TOKEN,
        "cd": cd # 선택) 최대 5자리, non(default): 시도정보, 2자리: 시군구, 5자리: 읍면동
        # "pg_yn": , # 선택) 경계 미포함(default): 0, 경계 포함: 1
    }

    temp_res = requests.get(
        url="https://sgisapi.kostat.go.kr/OpenAPI3/addr/stage.json",
        params=PARAMS
    )

    temp_sido_info_list = temp_res.json()["result"]

    return temp_sido_info_list

In [6]:
# 서울시에 속하는 구들의 이름을 리스트로
temp_seoul_info_list = get_si_info_list(consumer_key, consumer_secret, 11)
seoul_gu_name_list = [gu_name["addr_name"] for gu_name in temp_seoul_info_list ]
seoul_gu_name_list

['강남구',
 '강동구',
 '강북구',
 '강서구',
 '관악구',
 '광진구',
 '구로구',
 '금천구',
 '노원구',
 '도봉구',
 '동대문구',
 '동작구',
 '마포구',
 '서대문구',
 '서초구',
 '성동구',
 '성북구',
 '송파구',
 '양천구',
 '영등포구',
 '용산구',
 '은평구',
 '종로구',
 '중구',
 '중랑구']

In [24]:
# 반복문 초기값
START_INDEX = 1
END_INDEX = 1000
request_code = "INFO-000"
temp_result_list = []

while True:
    if(request_code!="INFO-000"):
        break
    elif(request_code=="INFO-000"):
        # API 조회간격 조절
        time.sleep(0.2) # 0.2초 대기후 요청

    # 요청 파라미터
    KEY = "614f614e5a6a303535387977726a57" # 서울 열린데이터 광장의 본인의 인증키
    TYPE = "json"
    SERVICE = "IndividuallyPostedLandPriceService"
    START_INDEX = START_INDEX + 1000
    END_INDEX = END_INDEX + 1000
    SIGUNGU_NM = "성동구"
    YEAR = 2020

    url = f"http://openapi.seoul.go.kr:8088/{KEY}/{TYPE}/{SERVICE}/{START_INDEX}/{END_INDEX}/{SIGUNGU_NM}/ / / / /{YEAR}"

    try:
        temp_res = requests.get(
            url=url
        )

        request_code = temp_res.json()["IndividuallyPostedLandPriceService"]["RESULT"]["CODE"]

        temp_res_list = temp_res.json()["IndividuallyPostedLandPriceService"]["row"]
        temp_result_list = temp_res_list + temp_result_list

    except:
        request_code = "INFO-200"


print(f"총 데이터수: {len(temp_result_list)}")

총 데이터수: 25671


In [25]:
sungdong_gu_giga_df = pd.DataFrame(temp_result_list)
sungdong_gu_giga_df.head()

,SIGUNGU_NM,SIGUNGU_CD,BJDONG_NM,BJDONG_CD,BONBEON,BUBEON,PILGI_NM,PILGI_CD,BASE_MON,JIGA,YEAR
0,성동구,11200,용답동,12200,0047,0016,토지,1,2020-01-01,3100000,2020
1,성동구,11200,용답동,12200,0047,0015,토지,1,2020-01-01,3100000,2020
2,성동구,11200,용답동,12200,0047,0014,토지,1,2020-01-01,4334000,2020
3,성동구,11200,용답동,12200,0047,0013,토지,1,2020-01-01,1023000,2020
4,성동구,11200,용답동,12200,0047,0012,토지,1,2020-01-01,4334000,2020


In [26]:
sungdong_gu_giga_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25671 entries, 0 to 25670
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   SIGUNGU_NM  25671 non-null  object
 1   SIGUNGU_CD  25671 non-null  object
 2   BJDONG_NM   25671 non-null  object
 3   BJDONG_CD   25671 non-null  object
 4   BONBEON     25671 non-null  object
 5   BUBEON      25671 non-null  object
 6   PILGI_NM    25671 non-null  object
 7   PILGI_CD    25671 non-null  object
 8   BASE_MON    25671 non-null  object
 9   JIGA        25671 non-null  object
 10  YEAR        25671 non-null  object
dtypes: object(11)
memory usage: 2.2+ MB


In [27]:
sungdong_gu_giga_df["JIGA"] = sungdong_gu_giga_df["JIGA"].astype(int)

In [28]:
sungdong_gu_dong_giga_df = sungdong_gu_giga_df.groupby(["BJDONG_NM"])["JIGA"].mean().to_frame().reset_index()
sungdong_gu_dong_giga_df # to_frame은 Series를 DataFrame으로 변환, reset_index()는 인덱스를 초기화

,BJDONG_NM,JIGA
0,금호동1가,2.765562e+06
1,금호동2가,3.496425e+06
2,금호동3가,3.808270e+06
3,금호동4가,3.783610e+06
4,도선동,5.473955e+06
5,마장동,3.356850e+06
6,사근동,2.296565e+06
7,성수동1가,4.912384e+06
8,성수동2가,5.512291e+06
9,송정동,2.768494e+06


In [30]:
# 성동구 정보를 SGIS에서 가져와서 GeoPandas를 이용해 GeoDataFrame을 생성
accessToken = get_access_token(consumer_key , consumer_secret )
params={
    "accessToken": accessToken,
    "year": 2020,
    "adm_cd": 11040, # 성동구 코드
    "low_search": 1
}

temp_res = requests.get(
    url="https://sgisapi.kostat.go.kr/OpenAPI3/boundary/hadmarea.geojson",
    params=params
)

In [31]:
sungdong_gu_dong_gpd = gpd.GeoDataFrame().from_features(temp_res.json())
sungdong_gu_dong_gpd.head()

,geometry,adm_cd,adm_nm,y,x
0,"POLYGON ((958262.940 1951830.159, 958806.169 1...",1104052,서울특별시 성동구 왕십리2동,1951605,958298
1,"POLYGON ((959182.984 1952644.101, 959214.539 1...",1104054,서울특별시 성동구 마장동,1952095,959431
2,"POLYGON ((960102.858 1951876.095, 960114.175 1...",1104055,서울특별시 성동구 사근동,1951065,959858
3,"POLYGON ((959261.179 1951611.886, 959269.656 1...",1104056,서울특별시 성동구 행당1동,1951176,959027
4,"POLYGON ((958545.562 1951320.264, 958584.077 1...",1104057,서울특별시 성동구 행당2동,1950913,958455


In [32]:
sungdong_gu_dong_gpd.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   geometry  17 non-null     geometry
 1   adm_cd    17 non-null     object  
 2   adm_nm    17 non-null     object  
 3   y         17 non-null     object  
 4   x         17 non-null     object  
dtypes: geometry(1), object(4)
memory usage: 812.0+ bytes


In [34]:
address_code_mappint_df = pd.read_csv("administration_zone_20201001.csv")
address_code_mappint_df.head()

,시도,시군구,행정구역명,행정동(행정기관명),법정동,행정구역코드,행정기관코드,행정기관 생성일,법정동코드,법정동 관할지역,행정동 영문명칭,비고,연계표 정비결과
0,서울특별시,서울특별시,서울특별시,서울특별시,서울특별시,11.0,1100000000,19880423,1100000000,NaN,Seoul,NaN,NaN
1,서울특별시,종로구,종로구,종로구,종로구,11010.0,1111000000,19880423,1111000000,NaN,Jongno-gu,NaN,NaN
2,서울특별시,종로구,청운효자동,청운효자동,청운동,1101072.0,1111051500,20081101,1111010100,NaN,Cheongunhyoja-dong,NaN,NaN
3,서울특별시,종로구,청운효자동,청운효자동,신교동,1101072.0,1111051500,20081101,1111010200,NaN,Cheongunhyoja-dong,NaN,NaN
4,서울특별시,종로구,청운효자동,청운효자동,궁정동,1101072.0,1111051500,20081101,1111010300,NaN,Cheongunhyoja-dong,NaN,NaN


In [35]:
address_code_mappint_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21687 entries, 0 to 21686
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   시도          21687 non-null  object 
 1   시군구         21687 non-null  object 
 2   행정구역명       21687 non-null  object 
 3   행정동(행정기관명)  21687 non-null  object 
 4   법정동         21687 non-null  object 
 5   행정구역코드      21608 non-null  float64
 6   행정기관코드      21687 non-null  int64  
 7   행정기관 생성일    21687 non-null  int64  
 8   법정동코드       21687 non-null  int64  
 9   법정동 관할지역    1704 non-null   object 
 10  행정동 영문명칭    21686 non-null  object 
 11  비고          673 non-null    object 
 12  연계표 정비결과    28 non-null     object 
dtypes: float64(1), int64(3), object(9)
memory usage: 2.2+ MB


In [36]:
address_code_mappint_df['행정구역코드'] = address_code_mappint_df['행정구역코드'].fillna(0).astype(int).astype(str)
# fillna(0) 은 결측치를 0으로 만듬

In [37]:
address_code_mappint_df.head()

,시도,시군구,행정구역명,행정동(행정기관명),법정동,행정구역코드,행정기관코드,행정기관 생성일,법정동코드,법정동 관할지역,행정동 영문명칭,비고,연계표 정비결과
0,서울특별시,서울특별시,서울특별시,서울특별시,서울특별시,11,1100000000,19880423,1100000000,NaN,Seoul,NaN,NaN
1,서울특별시,종로구,종로구,종로구,종로구,11010,1111000000,19880423,1111000000,NaN,Jongno-gu,NaN,NaN
2,서울특별시,종로구,청운효자동,청운효자동,청운동,1101072,1111051500,20081101,1111010100,NaN,Cheongunhyoja-dong,NaN,NaN
3,서울특별시,종로구,청운효자동,청운효자동,신교동,1101072,1111051500,20081101,1111010200,NaN,Cheongunhyoja-dong,NaN,NaN
4,서울특별시,종로구,청운효자동,청운효자동,궁정동,1101072,1111051500,20081101,1111010300,NaN,Cheongunhyoja-dong,NaN,NaN


In [38]:
address_code_mappint_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21687 entries, 0 to 21686
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   시도          21687 non-null  object
 1   시군구         21687 non-null  object
 2   행정구역명       21687 non-null  object
 3   행정동(행정기관명)  21687 non-null  object
 4   법정동         21687 non-null  object
 5   행정구역코드      21687 non-null  object
 6   행정기관코드      21687 non-null  int64 
 7   행정기관 생성일    21687 non-null  int64 
 8   법정동코드       21687 non-null  int64 
 9   법정동 관할지역    1704 non-null   object
 10  행정동 영문명칭    21686 non-null  object
 11  비고          673 non-null    object
 12  연계표 정비결과    28 non-null     object
dtypes: int64(3), object(10)
memory usage: 2.2+ MB


In [39]:
address_code_mappint_df = address_code_mappint_df[["시도", "시군구", "행정동(행정기관명)", "법정동", "행정구역코드"]]
address_code_mappint_df.head()

,시도,시군구,행정동(행정기관명),법정동,행정구역코드
0,서울특별시,서울특별시,서울특별시,서울특별시,11
1,서울특별시,종로구,종로구,종로구,11010
2,서울특별시,종로구,청운효자동,청운동,1101072
3,서울특별시,종로구,청운효자동,신교동,1101072
4,서울특별시,종로구,청운효자동,궁정동,1101072


In [40]:
# 칼럼명 한번에 변경하기
address_code_mappint_df.rename(
    columns={
        "시도": "si_do",
        "시군구": "si_gun_gu",
        "행정동(행정기관명)": "hj_nm",
        "법정동": "bj_nm",
        "행정구역코드": "adm_cd"
    },
    inplace=True
)

address_code_mappint_df.head()

,si_do,si_gun_gu,hj_nm,bj_nm,adm_cd
0,서울특별시,서울특별시,서울특별시,서울특별시,11
1,서울특별시,종로구,종로구,종로구,11010
2,서울특별시,종로구,청운효자동,청운동,1101072
3,서울특별시,종로구,청운효자동,신교동,1101072
4,서울특별시,종로구,청운효자동,궁정동,1101072


In [41]:
sungdong_gu_dong_gpd = pd.merge(sungdong_gu_dong_gpd, address_code_mappint_df, how="left", on="adm_cd")
# how="left"는 병합 방식을 지정, 여기서는 왼쪽 DataFrame의 모든 행을 유지하면서 오른쪽 DataFrame과 공통된 열("adm_cd")을 기준으로 병합, 오른쪽 DataFrame에 매칭되는 값이 없는 경우에는 누락된 값(NaN)으로 채웁니다.
sungdong_gu_dong_gpd.head()

,geometry,adm_cd,adm_nm,y,x,si_do,si_gun_gu,hj_nm,bj_nm
0,"POLYGON ((958262.940 1951830.159, 958806.169 1...",1104052,서울특별시 성동구 왕십리2동,1951605,958298,서울특별시,성동구,왕십리제2동,하왕십리동
1,"POLYGON ((959182.984 1952644.101, 959214.539 1...",1104054,서울특별시 성동구 마장동,1952095,959431,서울특별시,성동구,마장동,마장동
2,"POLYGON ((960102.858 1951876.095, 960114.175 1...",1104055,서울특별시 성동구 사근동,1951065,959858,서울특별시,성동구,사근동,사근동
3,"POLYGON ((960102.858 1951876.095, 960114.175 1...",1104055,서울특별시 성동구 사근동,1951065,959858,서울특별시,성동구,사근동,행당동
4,"POLYGON ((959261.179 1951611.886, 959269.656 1...",1104056,서울특별시 성동구 행당1동,1951176,959027,서울특별시,성동구,행당제1동,행당동


In [42]:
sungdong_gu_dong_gpd.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   geometry   23 non-null     geometry
 1   adm_cd     23 non-null     object  
 2   adm_nm     23 non-null     object  
 3   y          23 non-null     object  
 4   x          23 non-null     object  
 5   si_do      23 non-null     object  
 6   si_gun_gu  23 non-null     object  
 7   hj_nm      23 non-null     object  
 8   bj_nm      23 non-null     object  
dtypes: geometry(1), object(8)
memory usage: 1.7+ KB


In [44]:
sungdong_gu_dong_gpd = sungdong_gu_dong_gpd[["adm_cd", "adm_nm", "si_do", "si_gun_gu", "hj_nm", "bj_nm", "x", "y", "geometry"]]
sungdong_gu_dong_gpd.head()

,adm_cd,adm_nm,si_do,si_gun_gu,hj_nm,bj_nm,x,y,geometry
0,1104052,서울특별시 성동구 왕십리2동,서울특별시,성동구,왕십리제2동,하왕십리동,958298,1951605,"POLYGON ((958262.940 1951830.159, 958806.169 1..."
1,1104054,서울특별시 성동구 마장동,서울특별시,성동구,마장동,마장동,959431,1952095,"POLYGON ((959182.984 1952644.101, 959214.539 1..."
2,1104055,서울특별시 성동구 사근동,서울특별시,성동구,사근동,사근동,959858,1951065,"POLYGON ((960102.858 1951876.095, 960114.175 1..."
3,1104055,서울특별시 성동구 사근동,서울특별시,성동구,사근동,행당동,959858,1951065,"POLYGON ((960102.858 1951876.095, 960114.175 1..."
4,1104056,서울특별시 성동구 행당1동,서울특별시,성동구,행당제1동,행당동,959027,1951176,"POLYGON ((959261.179 1951611.886, 959269.656 1..."


In [45]:
# 좌표계 정의: UTM-K(epsg:5179)
sungdong_gu_dong_gpd.crs = {"init": "epsg:5179"}

# 좌표계 변환: UTM-K(epsg:5179) -> WSG84(epsg:4326)
sungdong_gu_dong_gpd = sungdong_gu_dong_gpd.to_crs({"init": "epsg:4326"})
sungdong_gu_dong_gpd.head()

C:\Users\김영준\PycharmProjects\aiffel_study\venv\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\김영준\PycharmProjects\aiffel_study\venv\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,adm_cd,adm_nm,si_do,si_gun_gu,hj_nm,bj_nm,x,y,geometry
0,1104052,서울특별시 성동구 왕십리2동,서울특별시,성동구,왕십리제2동,하왕십리동,958298,1951605,"POLYGON ((127.02740 37.56489, 127.03357 37.562..."
1,1104054,서울특별시 성동구 마장동,서울특별시,성동구,마장동,마장동,959431,1952095,"POLYGON ((127.03777 37.57227, 127.03813 37.572..."
2,1104055,서울특별시 성동구 사근동,서울특별시,성동구,사근동,사근동,959858,1951065,"POLYGON ((127.04823 37.56538, 127.04836 37.565..."
3,1104055,서울특별시 성동구 사근동,서울특별시,성동구,사근동,행당동,959858,1951065,"POLYGON ((127.04823 37.56538, 127.04836 37.565..."
4,1104056,서울특별시 성동구 행당1동,서울특별시,성동구,행당제1동,행당동,959027,1951176,"POLYGON ((127.03872 37.56297, 127.03881 37.562..."


In [46]:
# 인덱스를 동이름으로 변경
sungdong_gu_dong_gpd.set_index(
    "bj_nm",
    drop=False, # 해당 열을 인덱스로 지정하면서 기존의 열을 삭제할거면 True 유지할거면 False
    inplace=True
)

sungdong_gu_dong_gpd.head()

,adm_cd,adm_nm,si_do,si_gun_gu,hj_nm,bj_nm,x,y,geometry
bj_nm,,,,,,,,,
하왕십리동,1104052,서울특별시 성동구 왕십리2동,서울특별시,성동구,왕십리제2동,하왕십리동,958298,1951605,"POLYGON ((127.02740 37.56489, 127.03357 37.562..."
마장동,1104054,서울특별시 성동구 마장동,서울특별시,성동구,마장동,마장동,959431,1952095,"POLYGON ((127.03777 37.57227, 127.03813 37.572..."
사근동,1104055,서울특별시 성동구 사근동,서울특별시,성동구,사근동,사근동,959858,1951065,"POLYGON ((127.04823 37.56538, 127.04836 37.565..."
행당동,1104055,서울특별시 성동구 사근동,서울특별시,성동구,사근동,행당동,959858,1951065,"POLYGON ((127.04823 37.56538, 127.04836 37.565..."
행당동,1104056,서울특별시 성동구 행당1동,서울특별시,성동구,행당제1동,행당동,959027,1951176,"POLYGON ((127.03872 37.56297, 127.03881 37.562..."


In [49]:
sungdong_gu_dong_gpd[sungdong_gu_dong_gpd["bj_nm"] == "금호동1가"]

,adm_cd,adm_nm,si_do,si_gun_gu,hj_nm,bj_nm,x,y,geometry
bj_nm,,,,,,,,,
금호동1가,1104059,서울특별시 성동구 금호1가동,서울특별시,성동구,금호1가동,금호동1가,958067,1950570,"POLYGON ((127.02441 37.55790, 127.02462 37.557..."


In [50]:
center = [37.541, 126.986]
map = folium.Map(
    location=center,
    zoom_start=10,
    )

In [53]:
folium.Choropleth(
    geo_data=sungdong_gu_dong_gpd,  # 지리정보 데이터
    name="서울시 성동구 동별 평균 공시지가",  # Choropleth 레이어 이름
    data=sungdong_gu_dong_giga_df,  # 지도에 색상을 적용할 데이터
    columns=["BJDONG_NM", "JIGA"],  # BJDONG_NM은 지리정보와 연결되는 열, JIGA는 데이터 값을 나타내는 열
    key_on="feature.properties.bj_nm",  # 지리적 특성 정보를 지정
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    line_color='white',
    line_weight=0,
    highlight=False,  # 마우스 오버 효과 지정
    smooth_factor=1.0,  # 스무딩 효과 지정
    legend_name="평균 공시지가",  # 범례 이름 지정
).add_to(map)

In [52]:
map